<a href="https://colab.research.google.com/github/KerkPirr14/best-rate/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
from google.colab import drive

drive.mount("/content/drive")
df=pd.read_excel("/content/drive/My Drive/Исходящие вызовы абонента.xlsx")
df=df.drop_duplicates()

df["Дата звонка"]=pd.to_datetime(df["Дата звонка"], dayfirst=True)
df["Date_D"]= df["Дата звонка"].dt.to_period("D")
df["Date_M"]= df["Дата звонка"].dt.to_period("M")

# создание итогового ДФ и трех сортировочных ДФ (temp)
res, temp_D, temp_M, temp_mts=pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
res['Название тарифа'], res['Стоимость тарифа']='', ''

# Преобразование длительности разговора в минуты с округлением вверх
def sumtime_s(x):                   # функция принимает список из трех элементов
  return int(math.ceil((int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))/60))  # Функция посчитывает количество секунд, делит на 60, округляет вверх и возвращает значение ИНТ
for index in range(len(df['Длительность разговора'])): # перебор всех значений в столбце
  df['Длительность разговора'].iloc[index] = sumtime_s(df['Длительность разговора'].iloc[index].split(':')) # пребразование значения через фунцию, в фунцию передается список из трех значений СТР

# Подсчет минут в день и в месяц
temp_D['Длительность разговора'] = df.groupby("Date_D").sum()['Длительность разговора']
temp_M['Длительность разговора'] = df.groupby("Date_M").sum()['Длительность разговора']
temp_mts ['Длительность разговора'] = df[df['Оператор связи вызываемого абонента'] != "мтс"].groupby("Date_D").sum()['Длительность разговора']

# создание словаря тарифов (вместо if используется булева проверка на целочисленное деление)
tarif = {'Монстр общения': lambda x: round(1.5 + (x - 1 - (x - 9)*((x//10)>0)) * 0.5  + (x - 9)*((x//10)>0) * 1.0, 2),
         'Хочу сказать' : lambda x: round((x - (x - 5)*((x//6)>0)) * 3.95  + (x - 5)*((x//6)>0) * 0.4, 2),
         'Больше слов' : lambda x: round(555.0  + (x - 555)*((x//555)>0) * 1.95, 2),
         '33 копейки' : lambda x: round(1.0  + (x - 1)*((x//2)>0) * 0.33, 2),
         'Много звонков': lambda x: round((x - (x - 5 - (x - 30)*((x//30)>0))*((x//6)>0) - (x - 30)*((x//30)>0))*0.9 + (x - 5 - (x - 30)*((x//30)>0))*((x//6)>0) * 0.05  + (x - 30)*((x//30)>0) * 0.90, 2)}

# Добваление в итоговую таблицу тарифов с подсчетом сумм
res=res.append({'Название тарифа': 'Монстр общения', 'Стоимость тарифа': df['Длительность разговора'].map(tarif['Монстр общения']).sum()}, ignore_index = True)
res=res.append({'Название тарифа': 'Хочу сказать', 'Стоимость тарифа': temp_D['Длительность разговора'].map(tarif['Хочу сказать']).sum()}, ignore_index = True)
res=res.append({'Название тарифа': 'Больше слов', 'Стоимость тарифа': temp_M['Длительность разговора'].map(tarif['Больше слов']).sum()}, ignore_index = True)
res=res.append({'Название тарифа': '33 копейки', 'Стоимость тарифа': df['Длительность разговора'].map(tarif['33 копейки']).sum()}, ignore_index = True)
res=res.append({'Название тарифа': 'Много звонков', 'Стоимость тарифа': temp_mts['Длительность разговора'].map(tarif['Много звонков']).sum()}, ignore_index = True)

res




Mounted at /content/drive


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-1-f9d4ad377dda>:25: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  temp_D['Длительность разговора'] = df.groupby("Date_D").sum()['Длительность разговора']
<ipython-input-1-f9d4ad377dda>:26: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  temp_M['Длительность разговора'] = df.groupby("Date_M").sum()['Длител

,Название тарифа,Стоимость тарифа
0,Монстр общения,2465.50
1,Хочу сказать,3097.60
2,Больше слов,3712.20
3,33 копейки,1599.57
4,Много звонков,836.15
